In [15]:
import boto3
import json
import decimal

from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

In [2]:
# Get the service resource.
dynamodb = boto3.resource(
    'dynamodb'
    , aws_access_key_id='123'
    , aws_secret_access_key='123'
    , region_name='eu-west-1'
    , endpoint_url="http://192.168.99.100:8000"
)

## Step 1: Create a Table

In [9]:
table = dynamodb.create_table(
    TableName='Movies',
    AttributeDefinitions=[
        {
            'AttributeName': 'year',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'title',
            'AttributeType': 'S'
        },

    ],
    KeySchema=[
        {
            'AttributeName': 'year',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'title',
            'KeyType': 'RANGE'  # Sort key
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

table.meta.client.get_waiter('table_exists').wait(TableName='Movies')
print('Item count:', table.item_count)

Item count: 0


## Step 2: Load Sample Data

In [10]:
"""
moviedata.json

[
    {
      "year" : ... ,
      "title" : ... ,
      "info" : { ... }
    },
    {
      "year" : ...,
      "title" : ...,
      "info" : { ... }
    },

    ...
]
"""

with open("data/moviedata.json") as json_file:
    movies = json.load(json_file, parse_float = decimal.Decimal)
    for movie in movies:
        year = int(movie['year'])
        title = movie['title']
        info = movie['info']

        table.put_item(
           Item={
               'year': year,
               'title': title,
               'info': info,
            }
        )
        
print('Item count:', table.item_count)

Item count: 4609


## Step 3: Put, Update, and Delete an Item

In [11]:
# Helper class to convert a DynamoDB item to JSON

# The DecimalEncoder class is used to print out numbers stored using the Decimal class.
# The Boto SDK uses the Decimal class to hold DynamoDB number values.
class DecimalEncoder(json.JSONEncoder):

    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

In [ ]:
table = dynamodb.Table('Movies')

### Step 3.1: Add a New Item

In [13]:
title = "The Big New Movie"
year = 2015

In [32]:
response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':'Something happens.'
        }
    }
)

print(json.dumps(response, indent=4, cls=DecimalEncoder))

{
    "ResponseMetadata": {
        "RequestId": "73e1c535-2ae2-4854-9792-c312b70e1187",
        "HTTPStatusCode": 200
    }
}


#### Adding an Item conditionally
By default, the `put_item` method replaces any existing item with a new item. You can use a `ConditionExpression` parameter to avoid overwriting an existing item that has the same primary key.

In [33]:
try:
    table.put_item(
        Item={
            'year': year,
            'title': title,
            'info':{
                'plot':"Nothing happens at all.",
                'rating': decimal.Decimal(0)
            }
        },
        ConditionExpression=Attr("year").ne(year) & Attr("title").ne(title)
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("PutItem succeeded:")

response = table.get_item(
    Key={
        'year': year,
        'title': title
    }
)
item = response['Item']

print(json.dumps(item, indent=4, cls=DecimalEncoder))

The conditional request failed
{
    "title": "The Big New Movie",
    "year": 2015,
    "info": {
        "plot": "Something happens."
    }
}


Now remove the condition...

In [34]:
response = table.put_item(
    Item={
        'year': year,
        'title': title,
        'info':{
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(0)
        }
    }
)

response = table.get_item(
    Key={
        'year': year,
        'title': title
    }
)
item = response['Item']

print("PutItem succeeded:")
print(json.dumps(item, indent=4, cls=DecimalEncoder))

PutItem succeeded:
{
    "title": "The Big New Movie",
    "year": 2015,
    "info": {
        "rating": 0,
        "plot": "Nothing happens at all."
    }
}


### Step 3.2: Update an Item

You can update:
* values of existing attributes
* add new attributes
* remove attributes

#### Change the value of the existing attribute

In [35]:
response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="SET info.rating=:r, info.plot=:p, info.actors=:a",
    ExpressionAttributeValues={
        ':r': decimal.Decimal(5.5),
        ':p': "Everything happens all at once.",
        ':a': ["Larry", "Moe", "Curly"]
    },
    ReturnValues="UPDATED_NEW"  # the new versions of only the updated attributes are returned
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

UpdateItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "56d49ae6-534a-4883-850b-53513743e201",
        "HTTPStatusCode": 200
    },
    "Attributes": {
        "info": {
            "rating": 5.5,
            "plot": "Everything happens all at once.",
            "actors": [
                "Larry",
                "Moe",
                "Curly"
            ]
        }
    }
}


#### Increment an Atomic Counter

In [36]:
response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="SET info.rating = info.rating + :val",
    ExpressionAttributeValues={
        ':val': decimal.Decimal(1)
    },
    ReturnValues="UPDATED_NEW"
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

UpdateItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "e2937347-1352-4af0-add6-877fc1b8ee76",
        "HTTPStatusCode": 200
    },
    "Attributes": {
        "info": {
            "rating": 6.5,
            "plot": "Everything happens all at once.",
            "actors": [
                "Larry",
                "Moe",
                "Curly"
            ]
        }
    }
}


In [37]:
response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="SET info.rating = info.rating + :val",
    ExpressionAttributeValues={
        ':val': decimal.Decimal(1)
    },
    ReturnValues="UPDATED_NEW"
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

UpdateItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "848d04f7-ad62-4bfa-b653-648deb0154fd",
        "HTTPStatusCode": 200
    },
    "Attributes": {
        "info": {
            "rating": 7.5,
            "plot": "Everything happens all at once.",
            "actors": [
                "Larry",
                "Moe",
                "Curly"
            ]
        }
    }
}


#### Conditionally Update an Item

In [25]:
response = table.get_item(
    Key={
        'year': year,
        'title': title
    }
)
item = response['Item']

print(json.dumps(item, indent=4, cls=DecimalEncoder))

{
    "title": "The Big New Movie",
    "year": 2015,
    "info": {
        "rating": 7.5,
        "plot": "Everything happens all at once.",
        "actors": [
            "Larry",
            "Moe",
            "Curly"
        ]
    }
}


In [38]:
# Conditional update (will fail)
print("Attempting conditional update...")

try:
    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="REMOVE info.actors[0]",
        ConditionExpression="size(info.actors) > :num",
        ExpressionAttributeValues={
            ':num': 3
        },
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("UpdateItem succeeded:")
    print(json.dumps(response, indent=4, cls=DecimalEncoder))

Attempting conditional update...
The conditional request failed


Modify the code above so that the ConditionExpression looks like this:

`ConditionExpression="size(info.actors) >= :num",`

In [39]:
print("Attempting conditional update...")

try:
    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="REMOVE info.actors[0]",
        ConditionExpression="size(info.actors) >= :num",
        ExpressionAttributeValues={
            ':num': 3
        },
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("UpdateItem succeeded:")
    print(json.dumps(response, indent=4, cls=DecimalEncoder))

Attempting conditional update...
UpdateItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "7e56f7c9-2ffc-44e6-a4a9-e6df1460b34d",
        "HTTPStatusCode": 200
    },
    "Attributes": {
        "info": {
            "rating": 7.5,
            "plot": "Everything happens all at once.",
            "actors": [
                "Moe",
                "Curly"
            ]
        }
    }
}


### Step 3.3: Delete an Item

In [29]:
response = table.get_item(
    Key={
        'year': year,
        'title': title
    }
)
item = response['Item']

print(json.dumps(item, indent=4, cls=DecimalEncoder))

{
    "title": "The Big New Movie",
    "year": 2015,
    "info": {
        "rating": 7.5,
        "plot": "Everything happens all at once.",
        "actors": [
            "Moe",
            "Curly"
        ]
    }
}


Try to delete a specific movie item if its rating is 5 or less.

In [40]:
print("Attempting a conditional delete...")

try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        },
        ConditionExpression="info.rating <= :val",
        ExpressionAttributeValues= {
            ":val": decimal.Decimal(5)
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("DeleteItem succeeded:")
    print(json.dumps(response, indent=4, cls=DecimalEncoder))

Attempting a conditional delete...
The conditional request failed


Remove the condition...

In [41]:
response = table.delete_item(
    Key={
        'year': year,
        'title': title
    }
)

print("DeleteItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

DeleteItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "05a272ca-8b2c-4c10-b317-75bfd9d7b289",
        "HTTPStatusCode": 200
    }
}


## Step 4: Query and Scan the Data

* [`query`](http://boto3.readthedocs.org/en/latest/reference/services/dynamodb.html#DynamoDB.Table.query) retrieves data from a table
* [`scan`](http://boto3.readthedocs.org/en/latest/reference/services/dynamodb.html#DynamoDB.Table.scan) retrieves all of the table data

### Step 4.1: Query

Find all movies released in the `year` 1985.

In [51]:
print("Movies from 1985")

response = table.query(
    KeyConditionExpression=Key('year').eq(1985)
)

print('Number of movies found:', response['Count'], end='\n'*2)
print('The first 5 results:', end='\n')

for i in response['Items'][:5]:
    print(i['year'], ":", i['title'])

Movies from 1985
Number of movies found: 45

The first 5 results:
1985 : A Nightmare on Elm Street Part 2: Freddy's Revenge
1985 : A Room with a View
1985 : A View to a Kill
1985 : After Hours
1985 : Back to the Future


In [46]:
response.keys()

dict_keys(['Count', 'ScannedCount', 'Items', 'ResponseMetadata'])

**Note**

The Boto 3 SDK constructs a [Condition Expression](http://boto3.readthedocs.org/en/latest/reference/customizations/dynamodb.html#dynamodb-conditions) for you when you use the `Key` and `Attr` functions imported from `boto3.dynamodb.conditions`. You can also specify a Condition Expression as a string.

More information about [Condition Expressions](http://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Expressions.SpecifyingConditions.html#Expressions.SpecifyingConditions.ConditionExpressions).

In DynamoDB, you can optionally create one or more [**secondary indexes**](http://docs.aws.amazon.com/amazondynamodb/latest/developerguide/SecondaryIndexes.html) on a table, and query those indexes in the same way that you query a table. Secondary indexes give your applications additional flexibility by allowing queries on non-key attributes.

Find all movies in `year` 1992, with `title` beginning with the letter 'A' through the letter 'L'.

In [60]:
response = table.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    ExpressionAttributeNames={ "#yr": "year" }, # Expression Attribute Names for Projection Expression only.
    KeyConditionExpression=Key('year').eq(1992) & Key('title').between('A', 'L')
)

print('Items found:', response['Count'], end='\n'*2)
print(json.dumps(response['Items'][0], indent=4, cls=DecimalEncoder))

Items found: 28

{
    "title": "A Few Good Men",
    "year": 1992,
    "info": {
        "genres": [
            "Crime",
            "Drama",
            "Mystery",
            "Thriller"
        ],
        "actors": [
            "Tom Cruise"
        ]
    }
}


A [**projection expression**](http://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Expressions.AccessingItemAttributes.html) is a string that identifies the attributes you want.

**Note**<br>
`"year, title, info.genres, info.actors[0]"` projection expression would be invalid because `YEAR` is a [reserved word](http://docs.aws.amazon.com/amazondynamodb/latest/developerguide/ReservedWords.html).
To work around this, we define an *expression attribute name*. An expression attribute name is a placeholder that you use in the expression, as an alternative to the actual attribute name. An expression attribute name must begin with a #, followed by one alphabetic character, and then by zero or more alphanumeric characters.

### Step 4.2: Scan

The scan method reads **every item** in the entire table, and returns *all of the data* in the table. You can provide an optional filter_expression, so that only the items matching your criteria are returned. However, note that the *filter is only applied after* the entire table has been scanned.

In [65]:
table.item_count

4609

The code below scans the entire Movies table, which contains approximately 4,609 items. The scan specifies the optional filter to retrieve only the movies from the 1950s (approximately 100 items), and discard all of the others.

In [70]:
fe = Key('year').between(1950, 1959);
pe = "#yr, title, info.rating"
ean = { "#yr": "year", }  # Expression Attribute Names for Projection Expression only.


response = table.scan(
    FilterExpression=fe,
    ProjectionExpression=pe,
    ExpressionAttributeNames=ean
    )

print('Count', response['Count'])
print('ScannedCount', response['ScannedCount'])

print(json.dumps(response['Items'][0], indent=4, cls=DecimalEncoder))

Count 19
ScannedCount 2305
{
    "title": "High Noon",
    "year": 1952,
    "info": {
        "rating": 8.2
    }
}


In [67]:
response.keys()

dict_keys(['LastEvaluatedKey', 'Count', 'ScannedCount', 'Items', 'ResponseMetadata'])

In [68]:
response['LastEvaluatedKey']

{'title': 'Iron Man 2', 'year': Decimal('2010')}

In [71]:
while 'LastEvaluatedKey' in response:
    response = table.scan(
        ProjectionExpression=pe,
        FilterExpression=fe,
        ExpressionAttributeNames= ean,
        ExclusiveStartKey=response['LastEvaluatedKey']
        )
    print(response['Count'], response['ScannedCount'])   

54 2304


In [73]:
response.keys()

dict_keys(['Count', 'ScannedCount', 'Items', 'ResponseMetadata'])

In [74]:
response['ResponseMetadata']

{'HTTPStatusCode': 200, 'RequestId': 'bd1e2ffe-dd51-450c-987a-60f5136bebe2'}

### Step 5: (Optional) Delete the Table

In [75]:
table.delete()

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'RequestId': 'ec7dfed3-dedf-4ea9-97a3-57ed9e76ed92'},
 'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'year',
    'AttributeType': 'N'},
   {'AttributeName': 'title', 'AttributeType': 'S'}],
  'CreationDateTime': datetime.datetime(2016, 2, 9, 11, 48, 22, 111000, tzinfo=tzlocal()),
  'ItemCount': 4609,
  'KeySchema': [{'AttributeName': 'year', 'KeyType': 'HASH'},
   {'AttributeName': 'title', 'KeyType': 'RANGE'}],
  'ProvisionedThroughput': {'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/Movies',
  'TableName': 'Movies',
  'TableSizeBytes': 2095746,
  'TableStatus': 'ACTIVE'}}